### Show the position of ATP of predicted result

In [9]:
import os
import glob
import pandas as pd

# 指定你的文件夹路径
folder_path = 'predicted_test'  # 替换成你的文件夹路径

# 使用 glob 获取文件夹下所有 CSV 文件的完整路径
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))

# 创建一个列表保存每个文件的结果
results = []

# 遍历每个 CSV 文件
for file in csv_files:
    df = pd.read_csv(file)
    
    # 根据 'ATPseq Binding Sites' 列筛选出预测为ATP结合位点的行（这里假设标记 'B' 表示结合位点）
    atpseq_binding = df[df['ATPseq Binding Sites'] == 'B']
    
    # 提取位置编号（假设存储在 'NO' 列中），也可以根据需求提取其它信息
    positions = atpseq_binding['NO'].tolist()
    
    # 保存结果：可以保存文件名、预测的位点位置及个数等信息
    results.append({
        'filename': os.path.basename(file),
        'binding_positions': positions,
        'num_binding': len(positions)
    })

# 输出每个文件的结果，便于比较
for res in results:
    print(f"文件：{res['filename']}, 预测的ATP结合位点数量：{res['num_binding']}, 位置：{res['binding_positions']}")


文件：1mb9_chainB.csv, 预测的ATP结合位点数量：8, 位置：[243, 244, 246, 247, 248, 249, 266, 267]
文件：2j9c_chainA.csv, 预测的ATP结合位点数量：12, 位置：[9, 37, 38, 39, 41, 60, 88, 89, 90, 91, 92, 105]


### Show the position of ATP of experimental data

In [8]:
import os
import glob
from Bio.PDB import PDBParser, NeighborSearch
from Bio.PDB.Polypeptide import is_aa

# 指定存放 PDB 文件的文件夹路径（请替换为你的实际路径）
folder_path = 'raw_test'

# 获取文件夹下所有 pdb 文件
pdb_files = glob.glob(os.path.join(folder_path, '*.pdb'))

# 初始化 PDBParser
parser = PDBParser(QUIET=True)

# 设置距离阈值（单位：Å）
threshold = 4.0

for pdb_file in pdb_files:
    structure = parser.get_structure(os.path.basename(pdb_file), pdb_file)

    # 查找所有 ATP 配体的原子（ATP一般以 HETATM 记录出现）
    atp_atoms = []
    for model in structure:
        for chain in model:
            for residue in chain:
                if residue.get_resname() == "ATP":
                    for atom in residue:
                        atp_atoms.append(atom)

    # 如果没有找到 ATP 配体，则认为无结核点位
    if not atp_atoms:
        print(f"文件：{os.path.basename(pdb_file)}, 无结核点位（未找到 ATP 配体）。")
        continue

    # 收集所有标准氨基酸的原子
    protein_atoms = []
    for model in structure:
        for chain in model:
            for residue in chain:
                if is_aa(residue, standard=True):
                    protein_atoms.extend(list(residue.get_atoms()))

    # 构建邻域搜索树
    ns = NeighborSearch(protein_atoms)

    # 存储 ATP 结合点位对应的残基信息，格式为 (chain_id, residue_id, resname)
    binding_residues = set()
    for atp_atom in atp_atoms:
        close_atoms = ns.search(atp_atom.get_coord(), threshold)
        for atom in close_atoms:
            residue = atom.get_parent()
            if is_aa(residue, standard=True):
                chain_id = residue.get_parent().get_id()
                res_id = residue.get_id()  # 格式为 (hetfield, 序号, insertion code)
                binding_residues.add((chain_id, res_id, residue.get_resname()))

    # 提取所有结合点的残基序号（忽略链信息和三字母代码）
    positions = sorted([res_id[1] for _, res_id, _ in binding_residues])

    filename = os.path.basename(pdb_file)
    if binding_residues:
        count = len(binding_residues)
        print(f"文件：{filename}, 预测的ATP结合点位数量：{count}, 位置：{positions}")
    else:
        print(f"文件：{filename}, 无结核点位")


文件：1mb9_chainB.pdb, 预测的ATP结合点位数量：18, 位置：[247, 248, 249, 251, 252, 253, 254, 271, 272, 273, 330, 333, 346, 347, 348, 351, 423, 443]
文件：2j9c_chainA.pdb, 预测的ATP结合点位数量：13, 位置：[7, 35, 36, 37, 38, 44, 58, 86, 87, 88, 89, 90, 92]


### Extract ATP position from PDB file from PDB data bank

In [6]:
import os
import glob
import csv
from Bio.PDB import PDBParser, NeighborSearch
from Bio.PDB.Polypeptide import is_aa, three_to_one

# 指定存放 PDB 文件的文件夹路径（请替换为你的实际路径）
input_folder = 'extracted_chains'  # 输入PDB文件夹路径
# 指定存放生成的 CSV 文件的文件夹路径
output_folder = 'ATP_extracted'  # 输出CSV文件夹路径
os.makedirs(output_folder, exist_ok=True)

# 获取输入文件夹下所有 pdb 文件
pdb_files = glob.glob(os.path.join(input_folder, '*.pdb'))

# 初始化 PDBParser
parser = PDBParser(QUIET=True)

# 设置距离阈值（单位：Å）
threshold = 4.0

for pdb_file in pdb_files:
    # 解析 PDB 文件
    structure = parser.get_structure(os.path.basename(pdb_file), pdb_file)
    # 使用文件名（去除扩展名）作为 Prot.ID
    prot_id_base = os.path.splitext(os.path.basename(pdb_file))[0]

    # 1. 提取所有 ATP 配体的原子（ATP通常以 HETATM 记录出现）
    atp_atoms = []
    for model in structure:
        for chain in model:
            for residue in chain:
                if residue.get_resname() == "ATP":
                    atp_atoms.extend(list(residue.get_atoms()))
    
    # 2. 利用 ATP 配体的原子，利用邻域搜索找出 ATP 结合点位
    binding_residues = set()  # 保存格式：(chain_id, residue_id, 三字母残基代码)
    if atp_atoms:
        # 收集所有标准氨基酸的原子
        protein_atoms = []
        for model in structure:
            for chain in model:
                for residue in chain:
                    if is_aa(residue, standard=True):
                        protein_atoms.extend(list(residue.get_atoms()))
        ns = NeighborSearch(protein_atoms)
        for atp_atom in atp_atoms:
            close_atoms = ns.search(atp_atom.get_coord(), threshold)
            for atom in close_atoms:
                residue = atom.get_parent()
                if is_aa(residue, standard=True):
                    chain_id = residue.get_parent().get_id()
                    res_id = residue.get_id()  # 格式为 (hetfield, 序号, insertion code)
                    binding_residues.add((chain_id, res_id, residue.get_resname()))
    
    # 3. 遍历所有标准氨基酸残基，生成 CSV 的每一行
    rows = []
    header = ['Prot.ID', 'NO', 'Residue', 'ATP Binding Site']
    rows.append(header)
    
    for model in structure:
        for chain in model:
            chain_id = chain.get_id()
            for residue in chain:
                if not is_aa(residue, standard=True):
                    continue
                res_id = residue.get_id()  # (het, seq_number, insertion code)
                # 转换三字母代码为一字母代码
                try:
                    res_one = three_to_one(residue.get_resname())
                except Exception as e:
                    res_one = residue.get_resname()
                # 判断是否为 ATP 结合点位
                if (chain_id, res_id, residue.get_resname()) in binding_residues:
                    binding_flag = "B"
                else:
                    binding_flag = "N"
                row = [prot_id_base, res_id[1], res_one, binding_flag]
                rows.append(row)
    
    # 4. 写入 CSV 文件，文件名与 PDB 文件名对应
    output_file = os.path.join(output_folder, prot_id_base + '.csv')
    with open(output_file, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(rows)
    
    print(f"生成文件: {output_file}")

print("批量处理完成！")


生成文件: ATP_extracted\121p_chainA.csv
生成文件: ATP_extracted\1d4x_chainA.csv
生成文件: ATP_extracted\1f3f_chainC.csv
生成文件: ATP_extracted\1fit_chainA.csv
生成文件: ATP_extracted\1i58_chainA.csv
生成文件: ATP_extracted\1j09_chainA.csv
生成文件: ATP_extracted\1k90_chainA.csv
生成文件: ATP_extracted\1mb9_chainB.csv
生成文件: ATP_extracted\1rn8_chainA.csv
生成文件: ATP_extracted\1s1d_chainA.csv
生成文件: ATP_extracted\1to6_chainA.csv
生成文件: ATP_extracted\1twf_chainB.csv
生成文件: ATP_extracted\1un9_chainA.csv
生成文件: ATP_extracted\1vl1_chainA.csv
生成文件: ATP_extracted\1wc6_chainB.csv
生成文件: ATP_extracted\1xdn_chainA.csv
生成文件: ATP_extracted\1xdp_chainA.csv
生成文件: ATP_extracted\1yzy_chainA.csv
生成文件: ATP_extracted\1z0s_chainA.csv
生成文件: ATP_extracted\2aqx_chainB.csv
生成文件: ATP_extracted\2bz0_chainA.csv
生成文件: ATP_extracted\2f17_chainA.csv
生成文件: ATP_extracted\2i1o_chainA.csv
生成文件: ATP_extracted\2j9c_chainA.csv
生成文件: ATP_extracted\2jg1_chainC.csv
生成文件: ATP_extracted\2py7_chainX.csv
生成文件: ATP_extracted\2q16_chainB.csv
生成文件: ATP_extracted\2x14_cha

### compare residue

In [14]:
import os
import glob
import pandas as pd

# 真值CSV文件存放文件夹（例如提取后的文件）
gt_folder = 'ATP_extracted'
# 预测结果CSV文件存放文件夹
pred_folder = 'predicted_result'

# 获取真值文件夹下所有 CSV 文件
gt_files = glob.glob(os.path.join(gt_folder, '*.csv'))

# 遍历每个真值文件
for gt_file in gt_files:
    base_name = os.path.basename(gt_file)
    pred_file = os.path.join(pred_folder, base_name)
    
    if not os.path.exists(pred_file):
        print(f"预测结果文件不存在: {pred_file}")
        continue

    # 读取文件
    gt_df = pd.read_csv(gt_file)
    pred_df = pd.read_csv(pred_file)
    
    # 获取Residue列，转换为列表
    gt_residues = list(gt_df['Residue'])
    pred_residues = list(pred_df['Residue'])
    
    # 比较长度
    if len(gt_residues) != len(pred_residues):
        print(f"文件 {base_name}: 长度不匹配 -> 真值长度: {len(gt_residues)}, 预测长度: {len(pred_residues)}")
    
    # 检查是否完全匹配（仅对最短长度范围内比对）
    min_len = min(len(gt_residues), len(pred_residues))
    mismatches = []
    for i in range(min_len):
        if gt_residues[i] != pred_residues[i]:
            mismatches.append((i, gt_residues[i], pred_residues[i]))
    
    if mismatches:
        print(f"文件 {base_name}: Residue不匹配:")
        for idx, gt_res, pred_res in mismatches:
            print(f"  索引 {idx}: 真值 = {gt_res}, 预测 = {pred_res}")
    else:
        #if len(gt_residues) == len(pred_residues):
            #print(f"文件 {base_name}: Residue完全匹配。")
        #else:
            # 即使在最小长度内都匹配，但长度不同也说明存在缺失
        print(f"文件 {base_name}: 最短范围内Residue匹配，但整体长度不一致。")


文件 121p_chainA.csv: 最短范围内Residue匹配，但整体长度不一致。
文件 1d4x_chainA.csv: 最短范围内Residue匹配，但整体长度不一致。
文件 1f3f_chainC.csv: 最短范围内Residue匹配，但整体长度不一致。
文件 1fit_chainA.csv: 最短范围内Residue匹配，但整体长度不一致。
文件 1i58_chainA.csv: 最短范围内Residue匹配，但整体长度不一致。
文件 1j09_chainA.csv: 长度不匹配 -> 真值长度: 469, 预测长度: 468
文件 1j09_chainA.csv: 最短范围内Residue匹配，但整体长度不一致。
文件 1k90_chainA.csv: 最短范围内Residue匹配，但整体长度不一致。
文件 1mb9_chainB.csv: 最短范围内Residue匹配，但整体长度不一致。
文件 1rn8_chainA.csv: 最短范围内Residue匹配，但整体长度不一致。
文件 1s1d_chainA.csv: 最短范围内Residue匹配，但整体长度不一致。
文件 1to6_chainA.csv: 最短范围内Residue匹配，但整体长度不一致。
文件 1twf_chainB.csv: 最短范围内Residue匹配，但整体长度不一致。
文件 1un9_chainA.csv: 最短范围内Residue匹配，但整体长度不一致。
文件 1vl1_chainA.csv: 最短范围内Residue匹配，但整体长度不一致。
文件 1wc6_chainB.csv: 长度不匹配 -> 真值长度: 193, 预测长度: 192
文件 1wc6_chainB.csv: 最短范围内Residue匹配，但整体长度不一致。
文件 1xdn_chainA.csv: 最短范围内Residue匹配，但整体长度不一致。
文件 1xdp_chainA.csv: 最短范围内Residue匹配，但整体长度不一致。
文件 1yzy_chainA.csv: 最短范围内Residue匹配，但整体长度不一致。
文件 1z0s_chainA.csv: 最短范围内Residue匹配，但整体长度不一致。
文件 2aqx_chainB.csv: 最短范围内Residue匹配，但整体长度不一致。


### Compare and calculate TP...

In [12]:
import os
import glob
import pandas as pd
import math
from Bio import pairwise2

def align_labels(gt_residues, pred_residues, gt_binding, pred_binding):
    """
    对gt和pred的Residue序列进行全局比对，
    返回对齐后（两边均非gap）的真值结合标签和预测结合标签列表。
    """
    gt_seq = ''.join(gt_residues)
    pred_seq = ''.join(pred_residues)
    
    # 使用简单的全局比对（匹配得1分，不匹配得0分）
    alignments = pairwise2.align.globalxx(gt_seq, pred_seq)
    best = alignments[0]
    gt_aligned, pred_aligned = best.seqA, best.seqB

    aligned_gt_binding = []
    aligned_pred_binding = []
    i, j = 0, 0  # 指向原始序列的指针
    for a, b in zip(gt_aligned, pred_aligned):
        if a != '-' and b != '-':
            # 两边都有字符，则保存对应的结合标签
            aligned_gt_binding.append(gt_binding[i])
            aligned_pred_binding.append(pred_binding[j])
            i += 1
            j += 1
        elif a == '-' and b != '-':
            j += 1  # gt比对中出现缺失
        elif a != '-' and b == '-':
            i += 1  # pred比对中出现缺失
        else:
            # 同时为gap（很少见）
            pass
    return aligned_gt_binding, aligned_pred_binding

# 真值文件夹和预测结果文件夹（请修改为实际路径）
gt_folder = 'ATP_extracted'       # 存放提取后的真值CSV文件
pred_folder = 'predicted_result'          # 存放预测结果CSV文件

# 获取真值文件夹下所有CSV文件（假设每个蛋白对应一个CSV文件，且文件名一致）
gt_files = glob.glob(os.path.join(gt_folder, '*.csv'))

results = []

for gt_file in gt_files:
    base_name = os.path.basename(gt_file)
    pred_file = os.path.join(pred_folder, base_name)
    
    if not os.path.exists(pred_file):
        print(f"预测结果文件不存在: {pred_file}")
        continue

    # 读取真值和预测文件
    gt_df = pd.read_csv(gt_file)
    pred_df = pd.read_csv(pred_file)
    
    # 提取Residue序列（假设Residue列中的字符为一字母代码）
    gt_residues = list(gt_df['Residue'])
    pred_residues = list(pred_df['Residue'])
    
    # 提取真值的结合标签（假设列名为 "ATP Binding Site"，'B' 表示正，'N' 表示负）
    gt_binding = list(gt_df['ATP Binding Site'])
    
    # 对于两种预测方法分别计算指标
    for method, col in [('ATPseq', 'ATPseq Binding Sites'), ('ATPbind', 'ATPbind Binding Sites')]:
        pred_binding = list(pred_df[col])
        
        # 如果Residue序列不匹配，则使用序列对齐
        if gt_residues != pred_residues:
            aligned_gt_binding, aligned_pred_binding = align_labels(gt_residues, pred_residues, gt_binding, pred_binding)
        else:
            aligned_gt_binding = gt_binding
            aligned_pred_binding = pred_binding
        
        # 计算TP, FN, TN, FP（仅在对齐后的位置上计算）
        tp = sum(1 for gt, pred in zip(aligned_gt_binding, aligned_pred_binding) if gt == 'B' and pred == 'B')
        fn = sum(1 for gt, pred in zip(aligned_gt_binding, aligned_pred_binding) if gt == 'B' and pred == 'N')
        tn = sum(1 for gt, pred in zip(aligned_gt_binding, aligned_pred_binding) if gt == 'N' and pred == 'N')
        fp = sum(1 for gt, pred in zip(aligned_gt_binding, aligned_pred_binding) if gt == 'N' and pred == 'B')
        
        sen = tp / (tp + fn) if (tp + fn) != 0 else 0
        spe = tn / (tn + fp) if (tn + fp) != 0 else 0
        acc = (tp + tn) / (tp + fn + tn + fp) if (tp + fn + tn + fp) != 0 else 0
        pre = tp / (tp + fp) if (tp + fp) != 0 else 0
        denominator = math.sqrt((tp + fn) * (tp + fp) * (tn + fn) * (tn + fp))
        mcc = (tp * tn - fn * fp) / denominator if denominator != 0 else 0
        
        # 获取Prot.ID，假设真值文件中"Prot.ID"列的值都相同，取第一个
        prot_id = gt_df['Prot.ID'].iloc[0] if not gt_df.empty else base_name
        
        results.append({
            'Prot.ID': prot_id,
            'Method': method,
            'TP': tp,
            'FN': fn,
            'TN': tn,
            'FP': fp,
            'Sen': sen,
            'Spe': spe,
            'Acc': acc,
            'Pre': pre,
            'MCC': mcc
        })

# 将所有蛋白的评估结果保存到一个CSV文件中
results_df = pd.DataFrame(results)
output_file = 'evaluation_results_ATPbind.csv'
results_df.to_csv(output_file, index=False)
print(f"所有文件处理完毕，结果已保存到 {output_file}")


所有文件处理完毕，结果已保存到 evaluation_results_ATPbind.csv


### Check whether there is ATP or not

In [9]:
import os
import glob
from Bio.PDB import PDBParser, NeighborSearch
from Bio.PDB.Polypeptide import is_aa

# 指定存放 PDB 文件的文件夹路径（请替换为你的实际路径）
folder_path = 'extracted_chains'  # 存放89个PDB文件的文件夹

# 获取文件夹下所有 pdb 文件
pdb_files = glob.glob(os.path.join(folder_path, '*.pdb'))

# 初始化 PDBParser
parser = PDBParser(QUIET=True)
# 设置距离阈值，单位：Å
threshold = 4.0

# 用于存储没有 ATP 结合点的 PDB 文件名的列表
no_binding_files = []

for pdb_file in pdb_files:
    structure = parser.get_structure(os.path.basename(pdb_file), pdb_file)
    
    # 1. 查找所有 ATP 配体的原子（ATP 通常以 HETATM 记录出现）
    atp_atoms = []
    for model in structure:
        for chain in model:
            for residue in chain:
                if residue.get_resname() == "ATP":
                    atp_atoms.extend(list(residue.get_atoms()))
                    
    # 如果没有找到 ATP 配体，则认为该文件没有 ATP 结合点
    if not atp_atoms:
        no_binding_files.append(os.path.basename(pdb_file))
        continue
    
    # 2. 收集所有标准氨基酸的原子
    protein_atoms = []
    for model in structure:
        for chain in model:
            for residue in chain:
                if is_aa(residue, standard=True):
                    protein_atoms.extend(list(residue.get_atoms()))
                    
    # 构建邻域搜索树
    ns = NeighborSearch(protein_atoms)
    
    # 3. 对每个 ATP 原子，搜索距离在阈值内的蛋白质原子，
    #    如果至少有一个标准氨基酸残基落在此范围内，则认为存在 ATP 结合点
    binding_residues = set()
    for atp_atom in atp_atoms:
        close_atoms = ns.search(atp_atom.get_coord(), threshold)
        for atom in close_atoms:
            residue = atom.get_parent()
            if is_aa(residue, standard=True):
                binding_residues.add(residue.get_id())
    
    if not binding_residues:
        no_binding_files.append(os.path.basename(pdb_file))

# 打印没有 ATP 结合点的 PDB 文件名列表
print(f"没有 ATP 结合点的 PDB 文件:(个数：{len(no_binding_files)})")
print(no_binding_files)


没有 ATP 结合点的 PDB 文件:(个数：61)
['121p_chainA.pdb', '1f3f_chainC.pdb', '1fit_chainA.pdb', '1i58_chainA.pdb', '1k90_chainA.pdb', '1rn8_chainA.pdb', '1s1d_chainA.pdb', '1to6_chainA.pdb', '1twf_chainB.pdb', '1un9_chainA.pdb', '1vl1_chainA.pdb', '1wc6_chainB.pdb', '1yzy_chainA.pdb', '2bz0_chainA.pdb', '2f17_chainA.pdb', '2i1o_chainA.pdb', '2jg1_chainC.pdb', '2q16_chainB.pdb', '2x14_chainA.pdb', '2xan_chainA.pdb', '3c1m_chainC.pdb', '3erc_chainC.pdb', '3f2b_chainA.pdb', '3jqm_chainB.pdb', '3ruv_chainD.pdb', '3vth_chainA.pdb', '3wgu_chainC.pdb', '4amf_chainA.pdb', '4crj_chainA.pdb', '4edk_chainA.pdb', '4lac_chainC.pdb', '4ru9_chainA.pdb', '4uxx_chainC.pdb', '4yvz_chainA.pdb', '5dd7_chainA.pdb', '5dgh_chainA.pdb', '5guf_chainA.pdb', '5trd_chainA.pdb', '5w51_chainE.pdb', '6a8p_chainB.pdb', '6b5k_chainA.pdb', '6c02_chainA.pdb', '6cau_chainA.pdb', '6ci7_chainC.pdb', '6fl4_chainA.pdb', '6ig2_chainD.pdb', '6p1p_chainA.pdb', '6r5d_chainA.pdb', '6sqz_chainD.pdb', '6t0v_chainB.pdb', '6txe_chainA.pdb', '6v

In [ ]:
import pandas as pd

# 读取 evaluation_results.csv 文件（请确保文件路径正确）
results_df = pd.read_csv('evaluation_results_ATPbind.csv')

# 筛选 TP 和 FN 同时为 0 的蛋白质
no_binding_proteins = results_df[(results_df['TP'] == 0) & (results_df['FN'] == 0)]

# 获取符合条件的蛋白质的 Prot.ID 列表
protein_list = no_binding_proteins['Prot.ID'].unique().tolist()

print(f"没有ATP结核点的蛋白质 (TP和FN同时为0):个数为{len(protein_list)}")
print(protein_list)


没有ATP结核点的蛋白质 (TP和FN同时为0):个数为61
['121p_chainA', '1f3f_chainC', '1fit_chainA', '1i58_chainA', '1k90_chainA', '1rn8_chainA', '1s1d_chainA', '1to6_chainA', '1twf_chainB', '1un9_chainA', '1vl1_chainA', '1wc6_chainB', '1yzy_chainA', '2bz0_chainA', '2f17_chainA', '2i1o_chainA', '2jg1_chainC', '2q16_chainB', '2x14_chainA', '2xan_chainA', '3c1m_chainC', '3erc_chainC', '3f2b_chainA', '3jqm_chainB', '3ruv_chainD', '3vth_chainA', '3wgu_chainC', '4amf_chainA', '4crj_chainA', '4edk_chainA', '4lac_chainC', '4ru9_chainA', '4uxx_chainC', '4yvz_chainA', '5dd7_chainA', '5dgh_chainA', '5guf_chainA', '5trd_chainA', '5w51_chainE', '6a8p_chainB', '6b5k_chainA', '6c02_chainA', '6cau_chainA', '6ci7_chainC', '6fl4_chainA', '6ig2_chainD', '6p1p_chainA', '6r5d_chainA', '6sqz_chainD', '6t0v_chainB', '6txe_chainA', '6vd0_chainA', '7alr_chainA', '7cqq_chainA', '7d8i_chainA', '7edz_chainC', '7fgg_chainA', '7uld_chainA', '7v0f_chainA', '7y7p_chainA', '8dcd_chainA']


In [11]:
# 1. 去除 ".pdb" 后缀
no_suffix = [filename.replace(".pdb", "") for filename in no_binding_files]

# 2. 转换为集合
set_no_suffix = set(no_suffix)
set_protein_list = set(protein_list)

# 3. 比较两个集合
# 判断是否完全相同
if set_no_suffix == set_protein_list:
    print("两份列表完全相同！")
else:
    print("两份列表不完全相同。")

# 只在 no_suffix 中存在的蛋白
only_in_no_suffix = set_no_suffix - set_protein_list
if only_in_no_suffix:
    print("仅在 no_suffix 中的蛋白：", only_in_no_suffix)

# 只在 protein_list 中存在的蛋白
only_in_protein_list = set_protein_list - set_no_suffix
if only_in_protein_list:
    print("仅在 protein_list 中的蛋白：", only_in_protein_list)

# 两者都包含的蛋白
common = set_no_suffix & set_protein_list
if common:
    print("两份列表中都包含的蛋白：", common)


两份列表完全相同！
两份列表中都包含的蛋白： {'5trd_chainA', '4lac_chainC', '6c02_chainA', '1s1d_chainA', '6vd0_chainA', '7v0f_chainA', '6sqz_chainD', '3wgu_chainC', '1un9_chainA', '1twf_chainB', '1wc6_chainB', '6b5k_chainA', '121p_chainA', '7uld_chainA', '1yzy_chainA', '3erc_chainC', '1f3f_chainC', '3vth_chainA', '3ruv_chainD', '2bz0_chainA', '6ci7_chainC', '3f2b_chainA', '4ru9_chainA', '5dgh_chainA', '8dcd_chainA', '2jg1_chainC', '6txe_chainA', '4uxx_chainC', '7edz_chainC', '6a8p_chainB', '1rn8_chainA', '5guf_chainA', '1fit_chainA', '2x14_chainA', '2q16_chainB', '6fl4_chainA', '1to6_chainA', '2f17_chainA', '6p1p_chainA', '4edk_chainA', '6ig2_chainD', '1vl1_chainA', '6t0v_chainB', '7d8i_chainA', '7cqq_chainA', '7fgg_chainA', '6cau_chainA', '7y7p_chainA', '1i58_chainA', '2i1o_chainA', '5dd7_chainA', '4amf_chainA', '1k90_chainA', '5w51_chainE', '4yvz_chainA', '3c1m_chainC', '7alr_chainA', '2xan_chainA', '4crj_chainA', '6r5d_chainA', '3jqm_chainB'}


# Compare with AlphaFold

In [4]:
import os
import glob
import pandas as pd

# 真值CSV文件存放文件夹（例如提取后的文件）
gt_folder = 'ATP_cif'
# 预测结果CSV文件存放文件夹
pred_folder = 'ATP_AlphaFold'

# 获取真值文件夹下所有 CSV 文件
gt_files = glob.glob(os.path.join(gt_folder, '*.csv'))

# 遍历每个真值文件
for gt_file in gt_files:
    base_name = os.path.basename(gt_file)
    pred_file = os.path.join(pred_folder, base_name)
    
    if not os.path.exists(pred_file):
        print(f"预测结果文件不存在: {pred_file}")
        continue

    # 读取文件
    gt_df = pd.read_csv(gt_file)
    pred_df = pd.read_csv(pred_file)
    
    # 获取Residue列，转换为列表
    gt_residues = list(gt_df['Residue'])
    pred_residues = list(pred_df['Residue'])
    
    # 比较长度
    if len(gt_residues) != len(pred_residues):
        print(f"文件 {base_name}: 长度不匹配 -> 真值长度: {len(gt_residues)}, 预测长度: {len(pred_residues)}")
    
    # 检查是否完全匹配（仅对最短长度范围内比对）
    min_len = min(len(gt_residues), len(pred_residues))
    mismatches = []
    for i in range(min_len):
        if gt_residues[i] != pred_residues[i]:
            mismatches.append((i, gt_residues[i], pred_residues[i]))
    
    if mismatches:
        print(f"文件 {base_name}: Residue不匹配:")
        for idx, gt_res, pred_res in mismatches:
            print(f"  索引 {idx}: 真值 = {gt_res}, 预测 = {pred_res}")
    else:
        #if len(gt_residues) == len(pred_residues):
            #print(f"文件 {base_name}: Residue完全匹配。")
        #else:
            # 即使在最小长度内都匹配，但长度不同也说明存在缺失
        print(f"文件 {base_name}: 最短范围内Residue匹配，但整体长度不一致。")


文件 121p_chainA.csv: 最短范围内Residue匹配，但整体长度不一致。
文件 1d4x_chainA.csv: 长度不匹配 -> 真值长度: 368, 预测长度: 375
文件 1d4x_chainA.csv: Residue不匹配:
  索引 0: 真值 = E, 预测 = C
  索引 1: 真值 = V, 预测 = D
  索引 2: 真值 = A, 预测 = D
  索引 3: 真值 = A, 预测 = E
  索引 4: 真值 = L, 预测 = V
  索引 5: 真值 = V, 预测 = A
  索引 6: 真值 = V, 预测 = A
  索引 7: 真值 = D, 预测 = L
  索引 8: 真值 = N, 预测 = V
  索引 9: 真值 = G, 预测 = V
  索引 10: 真值 = S, 预测 = D
  索引 11: 真值 = G, 预测 = N
  索引 12: 真值 = M, 预测 = G
  索引 13: 真值 = C, 预测 = S
  索引 14: 真值 = K, 预测 = G
  索引 15: 真值 = A, 预测 = M
  索引 16: 真值 = G, 预测 = C
  索引 17: 真值 = F, 预测 = K
  索引 20: 真值 = D, 预测 = F
  索引 21: 真值 = D, 预测 = A
  索引 22: 真值 = A, 预测 = G
  索引 23: 真值 = P, 预测 = D
  索引 24: 真值 = R, 预测 = D
  索引 26: 真值 = V, 预测 = P
  索引 27: 真值 = F, 预测 = R
  索引 28: 真值 = P, 预测 = A
  索引 29: 真值 = S, 预测 = V
  索引 30: 真值 = I, 预测 = F
  索引 31: 真值 = V, 预测 = P
  索引 32: 真值 = G, 预测 = S
  索引 33: 真值 = R, 预测 = I
  索引 34: 真值 = P, 预测 = V
  索引 35: 真值 = R, 预测 = G
  索引 36: 真值 = H, 预测 = R
  索引 37: 真值 = Q, 预测 = P
  索引 38: 真值 = G, 预测 = R
  索引 39: 真值 = V, 预测

In [7]:
import os
import glob
import math
import pandas as pd
from Bio import pairwise2

def compute_metrics(gt_labels, pred_labels):
    """
    给定两个等长列表，值为 'B'/'N'，
    计算 TP, FN, TN, FP 及 Sen, Spe, Acc, Pre, MCC。
    """
    tp = fn = tn = fp = 0
    for g, p in zip(gt_labels, pred_labels):
        if g == 'B' and p == 'B':
            tp += 1
        elif g == 'B' and p == 'N':
            fn += 1
        elif g == 'N' and p == 'N':
            tn += 1
        elif g == 'N' and p == 'B':
            fp += 1

    sen = tp / (tp + fn) if (tp + fn) != 0 else 0
    spe = tn / (tn + fp) if (tn + fp) != 0 else 0
    acc = (tp + tn) / (tp + fn + tn + fp) if (tp + fn + tn + fp) != 0 else 0
    pre = tp / (tp + fp) if (tp + fp) != 0 else 0
    denom = math.sqrt((tp + fn)*(tp + fp)*(tn + fn)*(tn + fp))
    mcc = (tp*tn - fn*fp)/denom if denom else 0
    return tp, fn, tn, fp, sen, spe, acc, pre, mcc

def align_and_compare(gt_residues, gt_labels, pred_residues, pred_labels):
    """
    用全局比对 (globalxx) 对齐两条 Residue 序列，忽略gap处的残基，
    并在对齐后统计对应位置的真值标签与预测标签，从而得到分类指标。
    """
    # 将列表形式的 Residue 拼成字符串
    gt_seq = "".join(gt_residues)
    pred_seq = "".join(pred_residues)

    # 全局比对：match=1, mismatch=0
    alignments = pairwise2.align.globalxx(gt_seq, pred_seq)
    best = alignments[0]
    gt_aligned, pred_aligned = best.seqA, best.seqB

    # 遍历对齐结果，只在双方都非gap时计入对比
    i = j = 0
    aligned_gt_labels = []
    aligned_pred_labels = []

    for a_char, b_char in zip(gt_aligned, pred_aligned):
        if a_char == '-' and b_char == '-':
            continue
        elif a_char == '-':
            # gt缺失
            j += 1
            continue
        elif b_char == '-':
            # pred缺失
            i += 1
            continue
        else:
            # 双方都有氨基酸
            aligned_gt_labels.append(gt_labels[i])
            aligned_pred_labels.append(pred_labels[j])
            i += 1
            j += 1

    return compute_metrics(aligned_gt_labels, aligned_pred_labels)

def extract_sequence_and_labels(df, residue_col="Residue", label_col="ATP Binding Site", no_col="NO"):
    """
    将 CSV 中的行按 NO 排序后提取 Residue 和对应的标签列表。
    假设 Residue 已是一字母。
    """
    # 按 NO 排序，以确保顺序一致
    df_sorted = df.sort_values(by=no_col)
    residues = df_sorted[residue_col].tolist()
    labels = df_sorted[label_col].tolist()
    return residues, labels

# ========== 主程序 ==========

# 1) 真值文件夹
gt_folder = "ATP_cif_full"
# 2) 预测文件夹
pred_folder = "ATP_AlphaFold"

# 找到真值文件列表
gt_files = glob.glob(os.path.join(gt_folder, "*.csv"))
results = []

for gt_file in gt_files:
    base_name = os.path.basename(gt_file)   # 如 "1d4x_chainA.csv"
    # 假设预测文件同名
    pred_file = os.path.join(pred_folder, base_name)
    
    if not os.path.exists(pred_file):
        print(f"预测文件不存在: {pred_file}")
        continue

    # 读取真值和预测
    gt_df = pd.read_csv(gt_file)
    pred_df = pd.read_csv(pred_file)

    # 提取Residue和标签序列
    gt_residues, gt_labels = extract_sequence_and_labels(gt_df)
    pred_residues, pred_labels = extract_sequence_and_labels(pred_df)

    # 全局对齐并计算指标
    tp, fn, tn, fp, sen, spe, acc, pre, mcc = align_and_compare(
        gt_residues, gt_labels, pred_residues, pred_labels
    )

    # 生成结果
    # Prot.ID 可以直接用 base_name 去掉后缀
    prot_id = os.path.splitext(base_name)[0]
    results.append({
        "Prot.ID": prot_id,
        "TP": tp,
        "FN": fn,
        "TN": tn,
        "FP": fp,
        "Sen": sen,
        "Spe": spe,
        "Acc": acc,
        "Pre": pre,
        "MCC": mcc
    })

# 写结果到 CSV
results_df = pd.DataFrame(results)
results_df.to_csv("evaluation_results.csv", index=False)
print("所有蛋白处理完成，结果已保存到 evaluation_results_AlphaFold&Cif.csv")


所有蛋白处理完成，结果已保存到 evaluation_results_AlphaFold&Cif.csv


# Compare with AlphaFold with full cif

In [4]:
import os
import glob
import pandas as pd

# 真值CSV文件存放文件夹（例如提取后的文件）
gt_folder = 'ATP_cif_full'
# 预测结果CSV文件存放文件夹
pred_folder = 'ATP_AlphaFold'

# 获取真值文件夹下所有 CSV 文件
gt_files = glob.glob(os.path.join(gt_folder, '*.csv'))

# 遍历每个真值文件
for gt_file in gt_files:
    base_name = os.path.basename(gt_file)
    pred_file = os.path.join(pred_folder, base_name)
    
    if not os.path.exists(pred_file):
        print(f"预测结果文件不存在: {pred_file}")
        continue

    # 读取文件
    gt_df = pd.read_csv(gt_file)
    pred_df = pd.read_csv(pred_file)
    
    # 获取Residue列，转换为列表
    gt_residues = list(gt_df['Residue'])
    pred_residues = list(pred_df['Residue'])
    
    # 比较长度
    if len(gt_residues) != len(pred_residues):
        print(f"文件 {base_name}: 长度不匹配 -> 真值长度: {len(gt_residues)}, 预测长度: {len(pred_residues)}")
    
    # 检查是否完全匹配（仅对最短长度范围内比对）
    min_len = min(len(gt_residues), len(pred_residues))
    mismatches = []
    for i in range(min_len):
        if gt_residues[i] != pred_residues[i]:
            mismatches.append((i, gt_residues[i], pred_residues[i]))
    
    if mismatches:
        print(f"文件 {base_name}: Residue不匹配:")
        #for idx, gt_res, pred_res in mismatches:
            #print(f"  索引 {idx}: 真值 = {gt_res}, 预测 = {pred_res}")
    #else:
        #if len(gt_residues) == len(pred_residues):
            #print(f"文件 {base_name}: Residue完全匹配。")
        #else:
            # 即使在最小长度内都匹配，但长度不同也说明存在缺失
        #print(f"文件 {base_name}: 最短范围内Residue匹配，但整体长度不一致。")


文件 3c1m_chainC.csv: Residue不匹配:
文件 3f2b_chainA.csv: 长度不匹配 -> 真值长度: 1212, 预测长度: 1041
文件 3f2b_chainA.csv: Residue不匹配:
文件 3wgu_chainC.csv: 长度不匹配 -> 真值长度: 1016, 预测长度: 65
文件 3wgu_chainC.csv: Residue不匹配:
文件 5w51_chainE.csv: 长度不匹配 -> 真值长度: 215, 预测长度: 155
文件 5w51_chainE.csv: Residue不匹配:
文件 6p1p_chainA.csv: 长度不匹配 -> 真值长度: 357, 预测长度: 354
文件 6p1p_chainA.csv: Residue不匹配:
文件 6vd0_chainA.csv: Residue不匹配:
文件 7alr_chainA.csv: 长度不匹配 -> 真值长度: 500, 预测长度: 451
文件 7alr_chainA.csv: Residue不匹配:


In [8]:
import os
import glob
import math
import pandas as pd

def compute_metrics(gt_col, pred_col):
    """
    给定两个等长的序列（值为 'B'/'N'），
    计算 TP, FN, TN, FP 及 Sen, Spe, Acc, Pre, MCC。
    """
    tp = fn = tn = fp = 0
    for g, p in zip(gt_col, pred_col):
        if g == 'B' and p == 'B':
            tp += 1
        elif g == 'B' and p == 'N':
            fn += 1
        elif g == 'N' and p == 'N':
            tn += 1
        elif g == 'N' and p == 'B':
            fp += 1

    sen = tp / (tp + fn) if (tp + fn) != 0 else 0
    spe = tn / (tn + fp) if (tn + fp) != 0 else 0
    acc = (tp + tn) / (tp + fn + tn + fp) if (tp + fn + tn + fp) != 0 else 0
    pre = tp / (tp + fp) if (tp + fp) != 0 else 0
    denom = math.sqrt((tp + fn)*(tp + fp)*(tn + fn)*(tn + fp))
    mcc = (tp*tn - fn*fp)/denom if denom else 0
    return tp, fn, tn, fp, sen, spe, acc, pre, mcc

# ========== 主程序 ==========

# 假设：
# 1) 真值文件夹：gt_folder
# 2) 预测文件夹：pred_folder
# 3) 文件名相同，例如 "8dcd_chainA.csv" 都有 NO 列
# 4) 仅在 NO 相同的位置进行合并，不需要匹配 Prot.ID

gt_folder = "ATP_cif_full"
pred_folder = "ATP_AlphaFold"

gt_files = glob.glob(os.path.join(gt_folder, "*.csv"))
results = []

for gt_file in gt_files:
    base_name = os.path.basename(gt_file)
    pred_file = os.path.join(pred_folder, base_name)

    if not os.path.exists(pred_file):
        print(f"预测文件不存在: {pred_file}")
        continue

    # 读取真值和预测
    gt_df = pd.read_csv(gt_file)
    pred_df = pd.read_csv(pred_file)

    # 用 NO 作为唯一键合并
    merged = pd.merge(gt_df, pred_df, on="NO", how="inner", suffixes=("_gt", "_pred"))

    if merged.empty:
        print(f"合并后没有匹配行: {base_name}")
        continue

    # 若需要跳过 Residue='-' 的行，可在真值和预测侧都做检查
    # 例如如果真值这边 Residue_gt=='-' 或预测这边 Residue_pred=='-' 就跳过
    condition = (merged["Residue_gt"] != '-') & (merged["Residue_pred"] != '-')
    filtered = merged[condition]
    if filtered.empty:
        print(f"全是缺失行 '-'，跳过: {base_name}")
        continue

    # 真值标签 = filtered["ATP Binding Site_gt"]
    # 预测标签 = filtered["ATP Binding Site_pred"]
    gt_label = filtered["ATP Binding Site_gt"]
    pred_label = filtered["ATP Binding Site_pred"]

    # 计算指标
    tp, fn, tn, fp, sen, spe, acc, pre, mcc = compute_metrics(gt_label, pred_label)

    # Prot.ID 可视需要保存，也可用文件名去掉后缀当ID
    prot_id = os.path.splitext(base_name)[0]  # 例如 "8dcd_chainA"
    results.append({
        "Prot.ID": prot_id,
        "TP": tp,
        "FN": fn,
        "TN": tn,
        "FP": fp,
        "Sen": sen,
        "Spe": spe,
        "Acc": acc,
        "Pre": pre,
        "MCC": mcc
    })

# 输出汇总
results_df = pd.DataFrame(results)
output_file = "evaluation_results_AlphaFold&cif_full.csv"
results_df.to_csv(output_file, index=False)
print(f"批量对比完成，结果已保存到 {output_file}")


批量对比完成，结果已保存到 evaluation_results_AlphaFold&cif_full.csv


# Test code

### 只输出非单链ATP结核

In [11]:
import os
import glob
from Bio.PDB import PDBParser, NeighborSearch
from Bio.PDB.Polypeptide import is_aa

# 指定存放 PDB 文件的文件夹路径（请替换为你的实际路径）
folder_path = 'raw_test'
# 利用 glob 获取文件夹下所有 pdb 文件
pdb_files = glob.glob(os.path.join(folder_path, '*.pdb'))

# 初始化 PDBParser
parser = PDBParser(QUIET=True)

# 设置距离阈值，单位为 Å
threshold = 4.0

# 用于存储所有文件的结果
all_results = {}

for pdb_file in pdb_files:
    # 解析 PDB 文件
    structure = parser.get_structure(os.path.basename(pdb_file), pdb_file)

    # 查找所有 ATP 配体的原子（注意：ATP一般以 HETATM 记录出现）
    atp_atoms = []
    for model in structure:
        for chain in model:
            for residue in chain:
                if residue.get_resname() == "ATP":
                    for atom in residue:
                        atp_atoms.append(atom)

    if not atp_atoms:
        print(f"在 {os.path.basename(pdb_file)} 中未找到 ATP 配体。")
        continue

    # 收集所有标准氨基酸的原子
    protein_atoms = []
    for model in structure:
        for chain in model:
            for residue in chain:
                if is_aa(residue, standard=True):
                    protein_atoms.extend(list(residue.get_atoms()))

    # 构建邻域搜索树
    ns = NeighborSearch(protein_atoms)

    # 存储 ATP 结合位点对应的残基信息（使用集合去重）
    binding_residues = set()
    for atp_atom in atp_atoms:
        close_atoms = ns.search(atp_atom.get_coord(), threshold)
        for atom in close_atoms:
            residue = atom.get_parent()
            if is_aa(residue, standard=True):
                # 获取链ID和残基信息，residue.get_id() 返回一个元组：(hetfield, resseq, icode)
                chain_id = residue.get_parent().get_id()
                res_id = residue.get_id()
                binding_residues.add((chain_id, res_id, residue.get_resname()))

    # 保存结果
    all_results[os.path.basename(pdb_file)] = binding_residues

    # 检查 ATP 结合点位是否全部来自同一链
    unique_chains = {chain_id for chain_id, _, _ in binding_residues}
    if len(unique_chains) == 1:
        # 如果所有 ATP 结合点都只在一个链上，则不输出该文件
        continue

    # 输出当前文件的结果
    print(f"文件 {os.path.basename(pdb_file)} 中ATP结合点位（距离ATP配体在 {threshold} Å 范围内的残基）：")
    if binding_residues:
        for chain_id, res_id, resname in sorted(binding_residues, key=lambda x: x[1][1]):
            icode = res_id[2].strip() if res_id[2].strip() else ''
            print(f"  链 {chain_id}: 残基 {resname} {res_id[1]} {icode}")
    else:
        print("  未找到ATP结合点位。")
    print("-" * 50)


文件 1mb9.pdb 中ATP结合点位（距离ATP配体在 4.0 Å 范围内的残基）：
  链 B: 残基 VAL 247 
  链 A: 残基 VAL 247 
  链 B: 残基 LEU 248 
  链 A: 残基 LEU 248 
  链 B: 残基 SER 249 
  链 A: 残基 SER 249 
  链 B: 残基 GLY 251 
  链 A: 残基 GLY 251 
  链 B: 残基 ILE 252 
  链 A: 残基 ILE 252 
  链 A: 残基 ASP 253 
  链 B: 残基 ASP 253 
  链 B: 残基 SER 254 
  链 A: 残基 SER 254 
  链 B: 残基 VAL 271 
  链 A: 残基 VAL 271 
  链 A: 残基 SER 272 
  链 B: 残基 SER 272 
  链 A: 残基 MET 273 
  链 B: 残基 MET 273 
  链 A: 残基 TYR 326 
  链 A: 残基 LEU 330 
  链 B: 残基 LEU 330 
  链 B: 残基 LEU 333 
  链 A: 残基 LEU 333 
  链 B: 残基 THR 346 
  链 A: 残基 THR 346 
  链 A: 残基 GLY 347 
  链 B: 残基 GLY 347 
  链 A: 残基 TYR 348 
  链 B: 残基 TYR 348 
  链 A: 残基 ASP 351 
  链 B: 残基 ASP 351 
  链 A: 残基 LYS 423 
  链 B: 残基 LYS 423 
  链 B: 残基 LYS 443 
  链 A: 残基 LYS 443 
--------------------------------------------------


In [ ]:
import os
import glob
from Bio.PDB import PDBParser, NeighborSearch
from Bio.PDB.Polypeptide import is_aa

# 指定存放 PDB 文件的文件夹路径（请替换为你的实际路径）
folder_path = 'your_folder_path'

# 获取文件夹下所有 pdb 文件
pdb_files = glob.glob(os.path.join(folder_path, '*.pdb'))

# 初始化 PDBParser
parser = PDBParser(QUIET=True)

# 设置距离阈值，单位：Å
threshold = 4.0

# 存储所有文件的结果
all_results = {}

for pdb_file in pdb_files:
    structure = parser.get_structure(os.path.basename(pdb_file), pdb_file)
    
    # 查找所有 ATP 配体的原子（通常以 HETATM 记录出现）
    atp_atoms = []
    for model in structure:
        for chain in model:
            for residue in chain:
                if residue.get_resname() == "ATP":
                    atp_atoms.extend(list(residue.get_atoms()))
    
    if not atp_atoms:
        print(f"在 {os.path.basename(pdb_file)} 中未找到 ATP 配体。")
        continue

    # 收集所有标准氨基酸的原子
    protein_atoms = []
    for model in structure:
        for chain in model:
            for residue in chain:
                if is_aa(residue, standard=True):
                    protein_atoms.extend(list(residue.get_atoms()))
    
    # 构建邻域搜索树
    ns = NeighborSearch(protein_atoms)
    
    # 存储 ATP 结合点位对应的残基信息（去重）
    binding_residues = set()
    for atp_atom in atp_atoms:
        close_atoms = ns.search(atp_atom.get_coord(), threshold)
        for atom in close_atoms:
            residue = atom.get_parent()
            if is_aa(residue, standard=True):
                # residue.get_id() 返回一个元组：(hetfield, seq_number, insertion_code)
                res_id = residue.get_id()
                binding_residues.add((res_id, residue.get_resname()))
    
    all_results[os.path.basename(pdb_file)] = binding_residues

    # 输出当前文件的结果，不显示链信息
    print(f"文件 {os.path.basename(pdb_file)} 中ATP结合位点（距离ATP配体在 {threshold} Å 范围内的残基）：")
    if binding_residues:
        # 按照残基序号排序输出
        for res_id, resname in sorted(binding_residues, key=lambda x: x[0][1]):
            seq_num = res_id[1]
            insertion = res_id[2].strip() if res_id[2].strip() else ''
            print(f"  残基 {resname} {seq_num} {insertion}")
    else:
        print("  未找到ATP结合点位。")
    print("-" * 50)


### AlphaFold with PDB

In [ ]:
import os
import glob
import math
import pandas as pd
from Bio import pairwise2

def parse_predicted_file(filepath):
    """
    解析新的预测文件，仅包含预测到的ATP结合位点。
    跳过前2行说明，从第3行起，每行格式形如: "13 G"
    返回一个 dict: {残基编号(int): 残基一字母(str)}
    """
    predicted_dict = {}
    with open(filepath, 'r') as f:
        lines = f.readlines()
    
    # 假设前两行是说明或分割线，真正数据从第3行开始
    for line in lines[2:]:
        line = line.strip()
        if not line:
            continue
        parts = line.split()
        # 例如 parts = ["13", "G"]
        if len(parts) == 2:
            try:
                res_no = int(parts[0])
                res_letter = parts[1].upper()  # 转大写以统一
                predicted_dict[res_no] = res_letter
            except ValueError:
                pass  # 如果转换出错，跳过该行
    return predicted_dict

def align_labels(gt_residues, pred_residues, gt_labels, pred_labels):
    """
    对 gt_residues 与 pred_residues 做全局比对，只在双方都非 gap 的位置上比较标签。
    返回对齐后（非gap位置）的真值标签与预测标签。
    """
    gt_seq = ''.join(gt_residues)
    pred_seq = ''.join(pred_residues)
    
    # 简单全局比对 (match=1, mismatch=0)
    alignments = pairwise2.align.globalxx(gt_seq, pred_seq)
    best = alignments[0]
    gt_aligned, pred_aligned = best.seqA, best.seqB
    
    aligned_gt_labels = []
    aligned_pred_labels = []
    i, j = 0, 0  # 指向原始序列的索引
    
    for a, b in zip(gt_aligned, pred_aligned):
        if a != '-' and b != '-':
            # 两边都有氨基酸
            aligned_gt_labels.append(gt_labels[i])
            aligned_pred_labels.append(pred_labels[j])
            i += 1
            j += 1
        elif a != '-' and b == '-':
            # gt 有字符，pred 是 gap
            i += 1
        elif a == '-' and b != '-':
            # gt 是 gap，pred 有字符
            j += 1
        else:
            # 同时为 gap
            pass
    return aligned_gt_labels, aligned_pred_labels

def compute_metrics(gt_labels, pred_labels):
    """
    给定对齐后的真值标签和预测标签 (B/N)，计算 TP, FN, TN, FP 及 Sen, Spe, Acc, Pre, MCC
    """
    tp = sum(1 for g, p in zip(gt_labels, pred_labels) if g == 'B' and p == 'B')
    fn = sum(1 for g, p in zip(gt_labels, pred_labels) if g == 'B' and p == 'N')
    tn = sum(1 for g, p in zip(gt_labels, pred_labels) if g == 'N' and p == 'N')
    fp = sum(1 for g, p in zip(gt_labels, pred_labels) if g == 'N' and p == 'B')
    
    sen = tp / (tp + fn) if (tp + fn) != 0 else 0
    spe = tn / (tn + fp) if (tn + fp) != 0 else 0
    acc = (tp + tn) / (tp + fn + tn + fp) if (tp + fn + tn + fp) != 0 else 0
    pre = tp / (tp + fp) if (tp + fp) != 0 else 0
    denom = math.sqrt((tp + fn)*(tp + fp)*(tn + fn)*(tn + fp))
    mcc = (tp*tn - fn*fp) / denom if denom else 0
    
    return tp, fn, tn, fp, sen, spe, acc, pre, mcc

# ========== 主程序部分 ==========
# ground_truth_folder: 真值CSV文件所在文件夹
# predicted_folder: 新的预测数据所在文件夹
ground_truth_folder = 'ATP_extracted'
predicted_folder = 'ATP_AlphaFold'

# 获取所有真值文件（CSV）
gt_files = glob.glob(os.path.join(ground_truth_folder, '*.csv'))
results = []

for gt_file in gt_files:
    base_name = os.path.splitext(os.path.basename(gt_file))[0]  # 例如 "1d4x_chainA"
    pred_file = os.path.join(predicted_folder, base_name + '.txt')
    
    if not os.path.exists(pred_file):
        print(f"预测文件不存在: {pred_file}")
        continue
    
    # 1. 读取真值CSV
    gt_df = pd.read_csv(gt_file)
    # 提取序列(Residue)与标签(ATP Binding Site)
    gt_residues = gt_df['Residue'].tolist()  # 氨基酸一字母
    gt_labels = gt_df['ATP Binding Site'].tolist()  # 'B' or 'N'
    # 还需要提取每个 Residue 对应的编号 NO
    gt_numbers = gt_df['NO'].tolist()  # 用于匹配新预测
    
    # 2. 解析新的预测文件，得到 predicted_dict: {NO: Residue_letter}
    predicted_dict = parse_predicted_file(pred_file)
    
    # 3. 构建完整预测序列与标签
    pred_residues = []
    pred_labels = []
    for (res_no, res_letter, gt_label) in zip(gt_numbers, gt_residues, gt_labels):
        # 如果该编号在 predicted_dict 中，则表示预测为结合位点 'B'
        # 否则为 'N'
        if res_no in predicted_dict:
            pred_residues.append(predicted_dict[res_no])  # 采用预测文件给出的残基字母
            pred_labels.append('B')
        else:
            pred_residues.append(res_letter)  # 无论如何要给它一个字母，这里用真值的
            pred_labels.append('N')
    
    # 4. 如有需要，可进行序列对齐（如果 Residue 列不完全对应）
    # 如果 gt_residues == pred_residues，则可直接对比
    if gt_residues != pred_residues:
        aligned_gt_labels, aligned_pred_labels = align_labels(gt_residues, pred_residues, gt_labels, pred_labels)
    else:
        aligned_gt_labels = gt_labels
        aligned_pred_labels = pred_labels
    
    # 5. 计算指标
    tp, fn, tn, fp, sen, spe, acc, pre, mcc = compute_metrics(aligned_gt_labels, aligned_pred_labels)
    
    prot_id = gt_df['Prot.ID'].iloc[0] if not gt_df.empty else base_name
    results.append({
        'Prot.ID': prot_id,
        'TP': tp,
        'FN': fn,
        'TN': tn,
        'FP': fp,
        'Sen': sen,
        'Spe': spe,
        'Acc': acc,
        'Pre': pre,
        'MCC': mcc
    })

# 将结果保存为 evaluation_results_AlphaFold.csv
results_df = pd.DataFrame(results)
output_file = 'evaluation_results_AlphaFold.csv'
results_df.to_csv(output_file, index=False)
print(f"所有文件处理完毕，结果已保存到 {output_file}")
